In [21]:
# 1. Imports
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from torch.utils.data import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [22]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

full_dataset = datasets.ImageFolder("../../data", transform=transform)

keep_classes = ["Forest", "Residential"]
keep_indices = [i for i, (_, label) in enumerate(full_dataset) if full_dataset.classes[label] in keep_classes]
subset = Subset(full_dataset, keep_indices)

class_map = {
    full_dataset.class_to_idx["Forest"]: 0,
    full_dataset.class_to_idx["Residential"]: 1
}

class RelabeledDataset(Dataset):
    def __init__(self, subset, class_map):
        self.subset = subset
        self.class_map = class_map
    def __len__(self):
        return len(self.subset)
    def __getitem__(self, idx):
        x, y = self.subset[idx]
        return x, self.class_map[y]

dataset = RelabeledDataset(subset, class_map)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)


In [26]:
print(len(train_ds), len(val_ds))
print(full_dataset.classes)

import torch

all_labels = [y for _, y in dataset]
unique_labels = torch.unique(torch.tensor(all_labels))
print("Labels présents dans le dataset :", unique_labels.tolist())



4800 1200
['Forest', 'OOD', 'Residential']
Labels présents dans le dataset : [0, 1]


In [27]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 128), nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.fc(x)

model = CNNClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [28]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=30, patience=5):
    best_val_loss = float("inf")
    patience_counter = 0
    history = {"train_loss": [], "val_loss": [], "val_acc": []}

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)
        train_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_loss, correct = 0.0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = correct / len(val_loader.dataset)
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(best_model)
    return model, history


In [29]:
model = CNNClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model, history = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=30, patience=5)

Epoch [1/30] | Train Loss: 0.0431 | Val Loss: 0.0105 | Val Acc: 0.9950
Epoch [2/30] | Train Loss: 0.0093 | Val Loss: 0.0206 | Val Acc: 0.9925
Epoch [3/30] | Train Loss: 0.0103 | Val Loss: 0.0163 | Val Acc: 0.9950
Epoch [4/30] | Train Loss: 0.0055 | Val Loss: 0.0033 | Val Acc: 0.9992
Epoch [5/30] | Train Loss: 0.0033 | Val Loss: 0.0101 | Val Acc: 0.9958
Epoch [6/30] | Train Loss: 0.0070 | Val Loss: 0.0067 | Val Acc: 0.9975
Epoch [7/30] | Train Loss: 0.0009 | Val Loss: 0.0122 | Val Acc: 0.9975
Epoch [8/30] | Train Loss: 0.0041 | Val Loss: 0.0651 | Val Acc: 0.9875
Epoch [9/30] | Train Loss: 0.0070 | Val Loss: 0.0066 | Val Acc: 0.9983
Early stopping triggered.


CNN simple atteint quasi-100 % d’accuracy sur la validation avant l’arrêt précoce, donc il apprend parfaitement les deux classes et s’arrête correctement grâce à l’early stopping.

In [30]:
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, Subset, DataLoader
import torch

ood_base = ImageFolder("../../data/OOD", transform=transform)
orig_to_new = {
    ood_base.class_to_idx["Forest"]: 0,
    ood_base.class_to_idx["DenseResidential"]: 1,
    ood_base.class_to_idx["MediumResidential"]: 1,
}

class RelabeledDataset(Dataset):
    def __init__(self, base, mapping):
        self.base = base
        self.mapping = mapping
    def __len__(self):
        return len(self.base)
    def __getitem__(self, idx):
        x, y = self.base[idx]
        return x, self.mapping[y]

rel_ood = RelabeledDataset(ood_base, orig_to_new)

idx_dense  = [i for i,(_,y) in enumerate(ood_base.samples) if y == ood_base.class_to_idx["DenseResidential"]]
idx_medium = [i for i,(_,y) in enumerate(ood_base.samples) if y == ood_base.class_to_idx["MediumResidential"]]
idx_forest = [i for i,(_,y) in enumerate(ood_base.samples) if y == ood_base.class_to_idx["Forest"]]

ood_loader     = DataLoader(rel_ood, batch_size=64, shuffle=False)
dense_loader   = DataLoader(Subset(rel_ood, idx_dense),  batch_size=64, shuffle=False)
medium_loader  = DataLoader(Subset(rel_ood, idx_medium), batch_size=64, shuffle=False)
forest_loader  = DataLoader(Subset(rel_ood, idx_forest), batch_size=64, shuffle=False)

len(ood_base), len(idx_forest), len(idx_dense), len(idx_medium)


(40, 20, 10, 10)

In [31]:
def evaluate(model, loader):
    model.eval()
    tot = 0
    ok = 0
    cm = torch.zeros(2,2, dtype=torch.int64)
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            pred = logits.argmax(1)
            ok += (pred == y).sum().item()
            tot += y.size(0)
            for t,p in zip(y.cpu(), pred.cpu()):
                cm[t, p] += 1
    acc = ok / tot if tot else float("nan")
    return acc, cm

In [32]:
acc_all, cm_all       = evaluate(model, ood_loader)
acc_forest, cm_forest = evaluate(model, forest_loader)
acc_dense, cm_dense   = evaluate(model, dense_loader)
acc_medium, cm_medium = evaluate(model, medium_loader)

print("OOD global acc:", round(acc_all,4))
print("Forest acc:", round(acc_forest,4))
print("DenseResidential acc:", round(acc_dense,4))
print("MediumResidential acc:", round(acc_medium,4))
print("CM OOD:\n", cm_all.numpy())
print("CM Forest:\n", cm_forest.numpy())
print("CM Dense:\n", cm_dense.numpy())
print("CM Medium:\n", cm_medium.numpy())


OOD global acc: 0.775
Forest acc: 0.55
DenseResidential acc: 1.0
MediumResidential acc: 1.0
CM OOD:
 [[11  9]
 [ 0 20]]
CM Forest:
 [[11  9]
 [ 0  0]]
CM Dense:
 [[ 0  0]
 [ 0 10]]
CM Medium:
 [[ 0  0]
 [ 0 10]]


DenseResidential et MediumResidential sont parfaitement reconnues (100 %), mais elles sont très proches visuellement du domaine Residential d’entraînement.

Forest OOD est mal reconnu (55 %), avec de nombreux faux positifs vers la classe Residential.

Accuracy globale : 0.775, en forte baisse par rapport au ≈ 0.995 sur validation.

→ Conclusion : le modèle classique a sur-appris la distribution d’entraînement (textures, luminosité, pattern urbain précis). Sur les forêts d’un autre domaine, il ne généralise plus.